In [ ]:
from aavomics import database
import os
import pandas
import numpy
import anndata
import scanpy

from plotly import offline as plotly
from plotly import graph_objects
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly import graph_objects
from statsmodels.distributions.empirical_distribution import ECDF
import matplotlib.pyplot as plt
import dc_stat_think as dcst

In [ ]:
TRANSDUCTION_RATE_FILE_NAME = "aavomics_cell_type_transduction_rates.csv"
ANNDATA_FILE_NAME = "aavomics_mouse_cortex_2021.h5ad"
transduction_rate_df = pandas.read_csv(os.path.join(database.DATA_PATH, TRANSDUCTION_RATE_FILE_NAME), index_col=0)
adata = anndata.read_h5ad(os.path.join(database.DATA_PATH, ANNDATA_FILE_NAME))

In [ ]:
ALL_CELL_TYPES = ['Mature Oligodendrocytes','Red Blood Cells','Endothelial Cells',\
                    'Myoc- Astrocytes','Myoc+ Astrocytes',\
                    'L4/5','L6','L2/3','L5','L3','L5/6','L2',\
                    'Vip','Lamp5','Pax6','Sncg','Pvalb','Sst',\
                    'Committed Oligodendrocytes','OPCs','Pericytes','Vascular SMCs','Microglia','Perivascular Macrophages',\
                  'Leukocytes','VLMCs']


In [ ]:
three_dpi_conditions = ['20200903_TC8-AAV','20200904_TC9-AAV','20210728_TC12-AAV']
three_dpi_conditions_total_transduced_cells = []
for condition_index,condition in enumerate(three_dpi_conditions):   
    current_sample = transduction_rate_df.loc[condition]
    num_transduced_cells = 0
    for cell_index,cell in enumerate(ALL_CELL_TYPES):
        num_transduced_cells += current_sample.loc[cell+' Num Cells']*current_sample.loc[cell+" Transduction Rate"]
    three_dpi_conditions_total_transduced_cells.append(num_transduced_cells)

    
twentyfive_dpi_conditions = ['20190713_TC7-AAV','20201120_TC10-AAV','20210726_TC11-AAV']
twentyfive_dpi_conditions_total_transduced_cells = []
for condition_index,condition in enumerate(twentyfive_dpi_conditions):   
    current_sample = transduction_rate_df.loc[condition]
    num_transduced_cells = 0
    for cell_index,cell in enumerate(ALL_CELL_TYPES):
        if (pandas.isna(current_sample.loc[cell+' Num Cells']) or pandas.isna(current_sample.loc[cell+" Transduction Rate"])):
            continue
        num_transduced_cells += current_sample.loc[cell+' Num Cells']*current_sample.loc[cell+" Transduction Rate"]
    twentyfive_dpi_conditions_total_transduced_cells.append(num_transduced_cells)
    
    
CELLS_OF_INTEREST = ['Pericytes','Endothelial Cells','VLMCs','Vascular SMCs','Mature Oligodendrocytes','Myoc- Astrocytes','Myoc+ Astrocytes','Microglia','Committed Oligodendrocytes','OPCs','Perivascular Macrophages']

condition_cell_type_transduction_rates = {}  
condition_cell_type_transduction_rates["3DPI"] = {}
condition_cell_type_transduction_rates["25DPI"] = {}


for cell_index,cell in enumerate(CELLS_OF_INTEREST):
    
    current_cell_transduction_percents = []
    for condition_index,condition in enumerate(three_dpi_conditions):
        current_sample = transduction_rate_df.loc[condition]
        current_cell_transduction_percents.append(current_sample.loc[cell+" Transduction Rate"]*100)
    condition_cell_type_transduction_rates["3DPI"][cell] = current_cell_transduction_percents
    
    current_cell_transduction_percents = []
    for condition_index,condition in enumerate(twentyfive_dpi_conditions):
        current_sample = transduction_rate_df.loc[condition]
        current_cell_transduction_percents.append(current_sample.loc[cell+" Transduction Rate"]*100)
    condition_cell_type_transduction_rates["25DPI"][cell] = current_cell_transduction_percents
    

    
    
    

In [ ]:
# Note: neuron_st = neuron subtype

excitatory_neuron_subtype_trans_rate = ["L4/5 Transduction Rate","L2 Transduction Rate","L3 Transduction Rate",\
                                       "L6 Transduction Rate","L2/3 Transduction Rate","L5 Transduction Rate","L5/6 Transduction Rate"]
excitatory_neuron_subtype_num_cells = ["L4/5 Num Cells","L2 Num Cells","L3 Num Cells",\
                                       "L6 Num Cells","L2/3 Num Cells","L5 Num Cells","L5/6 Num Cells"]

current_condition_neuron_percents = []
for condition_index,condition in enumerate(three_dpi_conditions):
    current_sample = transduction_rate_df.loc[condition]
    
    current_neuron_transduced = 0
    current_neuron_counts = 0
    for neuron_st_index,neuron_subtype in enumerate(excitatory_neuron_subtype_trans_rate):
        if pandas.isna(current_sample.loc[neuron_subtype]) or pandas.isna(current_sample.loc[excitatory_neuron_subtype_num_cells[neuron_st_index]]):
            continue
        current_neuron_transduced += current_sample.loc[neuron_subtype]*current_sample.loc[excitatory_neuron_subtype_num_cells[neuron_st_index]]
        current_neuron_counts += current_sample.loc[excitatory_neuron_subtype_num_cells[neuron_st_index]]
    current_condition_neuron_percents.append(current_neuron_transduced/current_neuron_counts*100)

condition_cell_type_transduction_rates['3DPI']['Excitatory Neurons'] = current_condition_neuron_percents

current_condition_neuron_percents = []
for condition_index,condition in enumerate(twentyfive_dpi_conditions):
    current_sample = transduction_rate_df.loc[condition]
    current_neuron_transduced = 0
    current_neuron_counts = 0
    for neuron_st_index,neuron_subtype in enumerate(excitatory_neuron_subtype_trans_rate):
        if pandas.isna(current_sample.loc[neuron_subtype]) or pandas.isna(current_sample.loc[excitatory_neuron_subtype_num_cells[neuron_st_index]]):
            continue
        current_neuron_transduced += current_sample.loc[neuron_subtype]*current_sample.loc[excitatory_neuron_subtype_num_cells[neuron_st_index]]
        current_neuron_counts += current_sample.loc[excitatory_neuron_subtype_num_cells[neuron_st_index]]
    current_condition_neuron_percents.append(current_neuron_transduced/current_neuron_counts*100)

condition_cell_type_transduction_rates['25DPI']['Excitatory Neurons'] = current_condition_neuron_percents

inhibitory_neuron_subtype_trans_rate = ["Sst Transduction Rate","Lamp5 Transduction Rate",\
                                       "Vip Transduction Rate","Pax6 Transduction Rate",
                                       "Sncg Transduction Rate","Pvalb Transduction Rate"]
inhibitory_neuron_subtype_num_cells = ["Sst Num Cells","Lamp5 Num Cells",\
                                       "Vip Num Cells","Pax6 Num Cells",
                                       "Sncg Num Cells","Pvalb Num Cells"]

current_condition_neuron_percents = []

for condition_index,condition in enumerate(three_dpi_conditions):
    current_sample = transduction_rate_df.loc[condition]
    current_neuron_transduced = 0
    current_neuron_counts = 0
    for neuron_st_index,neuron_subtype in enumerate(inhibitory_neuron_subtype_trans_rate):
        if pandas.isna(current_sample.loc[neuron_subtype]) or pandas.isna(current_sample.loc[inhibitory_neuron_subtype_num_cells[neuron_st_index]]):
            continue
        current_neuron_transduced += current_sample.loc[neuron_subtype]*current_sample.loc[inhibitory_neuron_subtype_num_cells[neuron_st_index]]
        current_neuron_counts += current_sample.loc[inhibitory_neuron_subtype_num_cells[neuron_st_index]]
    current_condition_neuron_percents.append(current_neuron_transduced/current_neuron_counts*100)

    condition_cell_type_transduction_rates['3DPI']['Inhibitory Neurons'] = current_condition_neuron_percents
    
current_condition_neuron_percents = []
for condition_index,condition in enumerate(twentyfive_dpi_conditions):
    current_sample = transduction_rate_df.loc[condition]
    current_neuron_transduced = 0
    current_neuron_counts = 0
    for neuron_st_index,neuron_subtype in enumerate(inhibitory_neuron_subtype_trans_rate):
        if pandas.isna(current_sample.loc[neuron_subtype]) or pandas.isna(current_sample.loc[inhibitory_neuron_subtype_num_cells[neuron_st_index]]):
            continue
        current_neuron_transduced += current_sample.loc[neuron_subtype]*current_sample.loc[inhibitory_neuron_subtype_num_cells[neuron_st_index]]
        current_neuron_counts += current_sample.loc[inhibitory_neuron_subtype_num_cells[neuron_st_index]]
    current_condition_neuron_percents.append(current_neuron_transduced/current_neuron_counts*100)
    
    
    condition_cell_type_transduction_rates['25DPI']['Inhibitory Neurons'] = current_condition_neuron_percents

In [ ]:
CELL_TYPE_NAME_LABELS = {
    "Pericytes": ["Pericytes"],
    "Endothelial Cells": ["Endothelial Cells"],
    "VLMCs": ["VLMCs"],
    "Vascular SMCs": ["Vascular SMCs"],
    "Excitatory Neurons": ["L4/5", "L2", "L3", "L6", "L2/3", "L5", "L5/6"],
    "Inhibitory Neurons": ["Sncg", "Lamp5", "Pvalb", "Vip", "Sst", "Pax6"],
    "Myoc+ Astrocytes": ["Myoc+ Astrocytes"],
    "Myoc- Astrocytes": ["Myoc- Astrocytes"],
    "OPCs": ["OPCs"],
    "Committed Oligodendrocytes": ["Committed Oligodendrocytes"],
    "Mature Oligodendrocytes": ["Mature Oligodendrocytes"],
    "Microglia": ["Microglia"],
    "Perivascular Macrophages": ["Perivascular Macrophages"]
}


scatter_x_values = []
scatter_y_values = []

bar_x_values = []
bar_y_values = []

bar_colors = []

tick_text = []
tick_values = []

last_x_value = 0

for cell_type_name in list(CELL_TYPE_NAME_LABELS.keys()):
    
    values = []
    
    for condition in condition_cell_type_transduction_rates:
        
        x_value = last_x_value
        y_values = condition_cell_type_transduction_rates[condition][cell_type_name]
        
        scatter_x_values.extend([x_value]*len(y_values))
        scatter_y_values.extend(y_values)
        
        bar_x_values.append(x_value)
        bar_y_values.append(numpy.mean(y_values))
        
        if condition == "3DPI":
            bar_colors.append("red")
        else:
            bar_colors.append("magenta")
        
        tick_values.append(x_value)
        tick_text.append("%s %s" % (cell_type_name, condition))
        
        last_x_value += 1
        
    last_x_value += 0.5

traces = []

scatter = graph_objects.Scatter(
    x=scatter_x_values,
    y=scatter_y_values,
    mode="markers",
    marker={
        "color": "black"
    }
)

traces.append(scatter)

bar = graph_objects.Bar(
    x=bar_x_values,
    y=bar_y_values,
    marker={
        "color": bar_colors
    }
)

traces.append(bar)

layout = {
    "plot_bgcolor": "rgba(255, 255, 255, 0)",
    "paper_bgcolor": "rgba(255, 255, 255, 0)",
    "xaxis": {
        "tickvals": tick_values,
        "ticktext": tick_text
    },
    "yaxis": {
        "title": "Estimated Transduction Rate (%)"
    }
}
        
figure = graph_objects.Figure(data=traces, layout=layout)

plotly.iplot(figure)

figure.write_image("Figure_5A.svg")

In [ ]:
# NOTE: 
# Magenta is 25DPI and Red is 3DPI


pandas.concat(
    [pandas.DataFrame.from_dict(condition_cell_type_transduction_rates["3DPI"], orient="index"),
     pandas.DataFrame.from_dict(condition_cell_type_transduction_rates["25DPI"], orient="index")],
    axis=1
).to_csv('Figure_5A_transduction_rate.csv')